In [ ]:

def training_proc(num_training_updates,beta_i,lambd,model):
    
    model.train()
    
    train_res_recon_error = []
    train_res_perplexity = []
    
    for i in (xrange(num_training_updates)):
        (data, _) = next(iter(training_loader))
        data = data.to(device)
        optimizer.zero_grad()
        
        beta = beta_i 
        #beta = beta_i* ((i+1)/num_training_updates) 
        
        vq_loss, data_recon, perplexity,_,_ = model(data,beta,lambd)
        recon_error = F.mse_loss(data_recon, data) / data_variance
        loss = recon_error + vq_loss
        loss.backward()
    
        optimizer.step()
    
        train_res_recon_error.append(recon_error.item())
        train_res_perplexity.append(perplexity.item())
    
        if (i+1) % 100 == 0:
            print('%d iterations' % (i+1))
            print('recon_error: %.3f' % np.mean(train_res_recon_error[-100:]))
            print('perplexity: %.3f' % np.mean(train_res_perplexity[-100:]))
            print()
            
    return(train_res_recon_error,train_res_perplexity)

def test_proc(num_test_updates,beta,lambd,model): 
    test_res_recon_error = []
    test_res_perplexity = []
    
    model.eval()
    
    with torch.no_grad():

        for i in (xrange(num_test_updates)):
            (data, _) = next(iter(validation_loader))
            data = data.to(device)
            optimizer.zero_grad()
                
            vq_loss, data_recon, perplexity,_,_ = model(data,beta,lambd)
            recon_error = F.mse_loss(data_recon, data) / data_variance
            loss = recon_error + vq_loss    
            
            test_res_recon_error.append(recon_error.item())
            test_res_perplexity.append(perplexity.item())
        

    return(np.mean(test_res_recon_error),np.mean(perplexity.item()))


In [ ]:
num_training_updates = 4000


beta_i = 2
lambd = 0.001

recons_t = []
perps_t = []
models = []

#### test params:

beta_v = beta_i
num_test_updates = 50

recons_v = []
perps_v = []
betas = []



for k in tqdm(range(10)):
    if k == 0: 
        alpha = 0
    else: 
        alpha = 1
        
    beta_g = (alpha*beta_i**(k-1))
    
    model = Model(num_hiddens, num_residual_layers, num_residual_hiddens,
              num_embeddings, embedding_dim,
              commitment_cost, decay).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, amsgrad=False)
    
    recon_t,perp_t = training_proc(num_training_updates,beta_g,lambd,model)
    recons_t.append(recon_t)
    perps_t.append(perp_t)

    recon_v,perp_v = test_proc(num_test_updates,beta_g,lambd,model)
    recons_v.append(recon_v)
    print(recon_v)
    perps_v.append(perp_v)
    betas.append(alpha*beta_i**(k-1))
    



In [ ]:
cmap = plt.cm.coolwarm  # You can also use plt.cm.bwr for a similar effect
colors = cmap(1-np.linspace(0, 1, len(recons_t)))

f = plt.figure(figsize=(16,8))

# First subplot: Smoothed NMSE for all recon_v lists
ax1 = f.add_subplot(1, 2, 1)
for k,recon in enumerate(recons_t):
    train_res_recon_error_smooth = uniform_filter1d(recon, size=10, mode='nearest')
    beta_val = (betas[k]) #(torch.round(k*beta_i,decimals = 2).item())
    ax1.plot(train_res_recon_error_smooth, label = r'$\beta=$'+str(round(beta_val)),color=colors[k]) #'β ={:.2f}'.format(beta_val))

ax1.legend()
ax1.set_yscale('log') 
ax1.set_title('Smoothed NMSE.') 
ax1.set_xlabel('iteration') 

# Second subplot: Smoothed Average codebook usage (perplexity) for all perp_v lists
ax2 = f.add_subplot(1, 2, 2)
for k,perp in enumerate(perps_t):
    train_res_perplexity_smooth = uniform_filter1d(perp, size=10, mode='nearest')
    beta_val = betas[k]#(torch.round(k*beta_i,decimals = 2).item())
    ax2.plot(train_res_perplexity_smooth, label = r'$\beta=$'+str(beta_val),color=colors[k]) #'β={:.2f}'.format(beta_val))

ax2.legend()
ax2.set_title('Smoothed Average codebook usage (perplexity).')
ax2.set_xlabel('iteration')
#ax2.plot(betas_smooth, color='red', label='Beta')

plt.show()

In [ ]:
betas = [k for k in range(10)]

print(betas)
f = plt.figure(figsize=(14,5))
ax = f.add_subplot(1,2,1)
ax.scatter(betas,recons_v)
ax.plot(betas,recons_v,alpha = 0.3)
ax.set_yscale('log')
ax.set_title('Smoothed NMSE (test)')
ax.set_xlabel(r'$\log_2(\beta)$')

ax = f.add_subplot(1,2,2)
ax.scatter(betas,perps_v)
ax.plot(betas,perps_v,alpha = 0.3)
ax.set_title('Smoothed Average codebook usage (perplexity) (test)')
ax.set_xlabel(r'$\log_2(\beta)$')
